In [2]:
import numpy as np
import pandas as pd
import os 
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
import argparse
import wandb
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('using device:', device)

# # wandb
# wandb.init(project='cultural-rs', entity='armornine')
# print('Initialized WandB')

# # parse args
# parser = argparse.ArgumentParser(description="Argparser for modifying the training settings")
# parser.add_argument("--logs_path", type=str, help="Path to save outputs")
# parser.add_argument("--dataset_path", type=str, help="Path to read datasets")

# args = parser.parse_args()

# DATASET_PATH = args.dataset_path
# LOGS_PATH = args.logs_path

DATASET_PATH = '/home/mila/a/armin.moradi/scratch/data/LFM_2b_seperated_final'
LOGS_PATH = '/home/mila/a/armin.moradi/CulturalDiscoverability/results/model_outputs_testing_ipynb/'

print('Parsed Arguments')
print('DATASET_PATH:', DATASET_PATH)
print('LOGS_PATH:', LOGS_PATH)
print('-'*50)

# load data

interactions = pd.read_csv(os.path.join(DATASET_PATH, '10k_sampled_interactions.csv'))
user_demographics = pd.read_csv(os.path.join(DATASET_PATH, '10k_sampled_users.csv'))
item_demographics = pd.read_csv(os.path.join(DATASET_PATH, 'item_demographics.csv'))

print('Loaded Data', 'interactions:', len(interactions), 'users', len(user_demographics), 'items', len(item_demographics))
print('-'*50)

interactions['count'] = 1

# reindex IDs

user_demographics['new_user_id'] = user_demographics.index
user_demographics = user_demographics.sample(n=50, random_state=99) # TODO: CHANGE TO COMPLETE
print('Sampled User Demographics:', len(user_demographics))

df_final = interactions.merge(user_demographics, on='user_id', how='inner')

print(len(df_final), 'len of sampled user interactions')

# add random negative samples

user_ids = df_final['user_id'].unique()
item_ids = interactions['track_id'].unique()

negative_samples = []
for user_id in user_ids:
    user_interacted_items = interactions[interactions['user_id'] == user_id]['track_id'].values
    user_negative_items = np.setdiff1d(item_ids, user_interacted_items)
    user_negative_items = np.random.choice(user_negative_items, size=len(user_interacted_items), replace=False)
    for negative_item in user_negative_items:
        negative_samples.append([user_id, negative_item, 0])

negative_interactions = pd.DataFrame(negative_samples, columns=['user_id', 'track_id', 'count'])
negative_df = negative_interactions.merge(user_demographics, on='user_id', how='inner')
df_final = pd.concat([df_final, negative_df], ignore_index=True)

print('Added Negative Samples', '~ Added Items:', len(df_final[df_final['count'] == 0]))
print('-'*50)

item_demographics['new_track_id'] = item_demographics.index
item_demographics = item_demographics[item_demographics['track_id'].isin(interactions['track_id'])]
item_demographics = item_demographics.sample(frac=1, random_state=99) #TODO: CHANGE TO COMPLETE

df_final = df_final.merge(item_demographics, on='track_id', how='inner')

print('DF FINAL - n rows:', len(df_final))
print('-'*50)

# preprocessing

labels = ['0-18', '18-30', '30-50', '50+']
bins = [0, 18, 30, 50, 100]
df_final['user_age'] = pd.cut(df_final['user_age'], bins=bins, labels=labels, right=False)

# replace user id with new_user_id and delete user_id
df_final = df_final.drop(columns=['user_id'])
df_final = df_final.rename(columns={'new_user_id':'user_id'})

# replace user id with new_user_id and delete user_id
df_final = df_final.drop(columns=['track_id'])
df_final = df_final.rename(columns={'new_track_id':'track_id'})

cols = df_final.columns.tolist()
cols = [cols[-1], cols[3]] + cols[0:3] + cols[4:-1]
df_final = df_final[cols]
print('Finished Pre-processing')
print('-'*50)

using device: cpu
Parsed Arguments
DATASET_PATH: /home/mila/a/armin.moradi/scratch/data/LFM_2b_seperated_final
LOGS_PATH: /home/mila/a/armin.moradi/CulturalDiscoverability/results/model_outputs_testing_ipynb/
--------------------------------------------------
Loaded Data interactions: 10520266 users 9856 items 6321172
--------------------------------------------------
Sampled User Demographics: 50
59397 len of sampled user interactions
Added Negative Samples ~ Added Items: 59397
--------------------------------------------------
DF FINAL - n rows: 118794
--------------------------------------------------
Finished Pre-processing
--------------------------------------------------


In [3]:
# shape data

y = df_final['count']
x = df_final.drop(['count'], axis=1)
encoder = OneHotEncoder(sparse=False)
columns_to_encode = x.columns[2:]
encoder.fit(x[columns_to_encode])
encoded_x = encoder.transform(x[columns_to_encode])
encoded_x = pd.DataFrame(encoded_x, columns=encoder.get_feature_names_out(columns_to_encode))
x = pd.concat([x[['user_id', 'track_id']], encoded_x], axis=1)

user_country_indices = [i for i, col in enumerate(x.columns) if col.startswith('user_country_')]
artist_country_indices = [i for i, col in enumerate(x.columns) if col.startswith('artist_country_')]
user_gender_indices = [i for i, col in enumerate(x.columns) if col.startswith('user_gender_')]
artist_gender_indices = [i for i, col in enumerate(x.columns) if col.startswith('artist_gender')]
user_age_indices = [i for i, col in enumerate(x.columns) if col.startswith('user_age_')]
unique_user_ids, unique_track_ids = max(x['user_id']) + 1, max(x['track_id']) + 1

# split the interactions of each user into train, val, and test (80-10-10)
train_share = 0.8
val_share = 0.1
test_share = 1 - train_share - val_share

user_interactions = {}
for i in range(len(x)):
    user_id = x.iloc[i]['user_id']
    if user_id not in user_interactions:
        user_interactions[user_id] = []
    user_interactions[user_id].append(i)

train_indices, val_indices, test_indices = [], [], []
for user_id, indices in user_interactions.items():
    np.random.shuffle(indices)
    train_indices += indices[:int(train_share * len(indices))]
    val_indices += indices[int(train_share * len(indices)):int((train_share + val_share ) * len(indices))]
    test_indices += indices[int((1 - test_share) * len(indices)):]
    
train_x, train_y = x.iloc[train_indices], y.iloc[train_indices]
val_x, val_y = x.iloc[val_indices], y.iloc[val_indices]
test_x, test_y = x.iloc[test_indices], y.iloc[test_indices]

# create dataloaders

train_dataset = TensorDataset(torch.tensor(train_x.values, dtype=torch.float), torch.tensor(train_y.values, dtype=torch.float))
val_dataset = TensorDataset(torch.tensor(val_x.values, dtype=torch.float), torch.tensor(val_y.values, dtype=torch.float))
test_dataset = TensorDataset(torch.tensor(test_x.values, dtype=torch.float), torch.tensor(test_y.values, dtype=torch.float))

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
print(f'created loaders with batchsize {batch_size}', len(train_dataset), len(val_dataset), len(test_dataset))

TypeError: __init__() got an unexpected keyword argument 'sparse'

In [78]:
# training model with only user_id and track_id and countries

class AgnosticNCF(nn.Module):
    def __init__(self, num_users, num_tracks, user_country_dim, artist_country_dim, hidden_size=[256, 128, 64]):
        super(AgnosticNCF, self).__init__()

        self.embedding_size = 32

        self.user_id_embedding = nn.Embedding(num_users, self.embedding_size)
        self.track_id_embedding = nn.Embedding(num_tracks, self.embedding_size)


        self.fc_layers = nn.Sequential(
            nn.Linear(self.embedding_size * 2, hidden_size[0]),
            nn.ReLU(),
            nn.Linear(hidden_size[0], hidden_size[1]),
            nn.ReLU(),
            nn.Linear(hidden_size[1], hidden_size[2]),
            nn.ReLU(),
            nn.Linear(hidden_size[2], 1),
        )

    def forward(self, user_id, artist_id, user_country, artist_country):

        user_id_embeds = self.user_id_embedding(user_id)
        track_id_embeds = self.track_id_embedding(artist_id)
        

        concatenated = torch.cat([user_id_embeds, track_id_embeds], dim=1)
        output = self.fc_layers(concatenated.float())

        return output.squeeze()

print('Start Training')
print('-'*50)

start_time = time.time()
model = AgnosticNCF(num_users=unique_user_ids, num_tracks=unique_track_ids,
                                    user_country_dim=len(user_country_indices),
                                    artist_country_dim=len(artist_country_indices),
                                    hidden_size=[64, 128, 64]).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 30

val_loss = []
train_loss = []


for epoch in tqdm(range(num_epochs), desc='Epochs'):
    model.train()
    total_loss = 0.0
    for xx, yy in train_loader:
        user_id = xx[:, 0].long().to(device)
        track_id = xx[:, 1].long().to(device)

        user_country = xx[:, user_country_indices].long().to(device)
        artist_country = xx[:, artist_country_indices].long().to(device)
        
        optimizer.zero_grad()
        outputs = model(user_id, track_id, user_country, artist_country)
        loss = criterion(outputs.float(), yy.float().to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    # wandb.log({'train_loss': total_loss / len(train_loader)})    
    model.eval()
    total_val_loss = 0.0
    for xx, yy in val_loader:
        user_id = xx[:, 0].long().to(device)
        track_id = xx[:, 1].long().to(device)

        user_country = xx[:, user_country_indices].long().to(device)
        artist_country = xx[:, artist_country_indices].long().to(device)

        outputs = model(user_id, track_id, user_country, artist_country)
        loss = criterion(outputs.float(), yy.float().to(device))
        total_val_loss += loss.item()
    # wandb.log({'val_loss': total_val_loss / len(val_loader)})        
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}, Val Loss: {total_val_loss / len(val_loader)}')
    train_loss.append(total_loss / len(train_loader))
    val_loss.append(total_val_loss / len(val_loader))

    # torch.save(model.state_dict(), LOGS_PATH + 'model_weights.pth')

print('Training Time:', round((time.time() - start_time)/60, 2), 'minutes')
print('-'*50)

Start Training
--------------------------------------------------


Epochs:   0%|          | 0/30 [21:15<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# training model with only user_id and track_id and countries

class NeuralCollaborativeFiltering(nn.Module):
    def __init__(self, num_users, num_tracks, user_country_dim, artist_country_dim, hidden_size=[256, 128, 64]):
        super(NeuralCollaborativeFiltering, self).__init__()

        self.embedding_size = 32

        self.user_id_embedding = nn.Embedding(num_users, self.embedding_size)
        self.track_id_embedding = nn.Embedding(num_tracks, self.embedding_size)


        self.fc_layers = nn.Sequential(
            nn.Linear(self.embedding_size * 2, hidden_size[0]),
            nn.ReLU(),
            nn.Linear(hidden_size[0], hidden_size[1]),
            nn.ReLU(),
            nn.Linear(hidden_size[1], hidden_size[2]),
            nn.ReLU(),
            nn.Linear(hidden_size[2], 1),
        )

    def forward(self, user_id, artist_id, user_country, artist_country):

        user_id_embeds = self.user_id_embedding(user_id)
        track_id_embeds = self.track_id_embedding(artist_id)
        
        # user_country_embeds = self.user_country_embedding(user_country)
        # artist_country_embeds = self.artist_country_embedding(artist_country)

        concatenated = torch.cat([user_id_embeds, track_id_embeds, user_country, artist_country], dim=1)
        output = self.fc_layers(concatenated.float())

        return output.squeeze()

print('Start Training')
print('-'*50)

start_time = time.time()
model = NeuralCollaborativeFiltering(num_users=unique_user_ids, num_tracks=unique_track_ids,
                                    user_country_dim=len(user_country_indices),
                                    artist_country_dim=len(artist_country_indices),
                                    hidden_size=[64, 128, 64]).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 30

val_loss = []
train_loss = []


for epoch in tqdm(range(num_epochs), desc='Epochs'):
    model.train()
    total_loss = 0.0
    for xx, yy in train_loader:
        user_id = xx[:, 0].long().to(device)
        track_id = xx[:, 1].long().to(device)

        user_country = xx[:, user_country_indices].long().to(device)
        artist_country = xx[:, artist_country_indices].long().to(device)
        
        optimizer.zero_grad()
        outputs = model(user_id, track_id, user_country, artist_country)
        loss = criterion(outputs.float(), yy.float().to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    # wandb.log({'train_loss': total_loss / len(train_loader)})    
    model.eval()
    total_val_loss = 0.0
    for xx, yy in val_loader:
        user_id = xx[:, 0].long().to(device)
        track_id = xx[:, 1].long().to(device)

        user_country = xx[:, user_country_indices].long().to(device)
        artist_country = xx[:, artist_country_indices].long().to(device)

        outputs = model(user_id, track_id, user_country, artist_country)
        loss = criterion(outputs.float(), yy.float().to(device))
        total_val_loss += loss.item()
    # wandb.log({'val_loss': total_val_loss / len(val_loader)})        
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}, Val Loss: {total_val_loss / len(val_loader)}')
    train_loss.append(total_loss / len(train_loader))
    val_loss.append(total_val_loss / len(val_loader))

    # torch.save(model.state_dict(), LOGS_PATH + 'model_weights.pth')

print('Training Time:', round((time.time() - start_time)/60, 2), 'minutes')
print('-'*50)

Start Training
--------------------------------------------------


Epochs:   3%|▎         | 1/30 [00:22<11:06, 22.98s/it]

Epoch 1/30, Loss: 0.3442390374839306, Val Loss: 0.2524240016937256


Epochs:   7%|▋         | 2/30 [00:44<10:23, 22.25s/it]

Epoch 2/30, Loss: 0.2538522370159626, Val Loss: 0.26560235023498535


...

Epoch 14/30, Loss: 0.13804347347468138, Val Loss: 0.274686723947525


Epochs:  50%|█████     | 15/30 [05:41<05:52, 23.50s/it]

Epoch 15/30, Loss: 0.1358207892626524, Val Loss: 0.27396443486213684


Epochs:  50%|█████     | 15/30 [06:00<06:00, 24.04s/it]


KeyboardInterrupt: 

In [ ]:
# test

model.eval()
total_test_loss = 0.0
diff_list = []

with torch.no_grad():
    for xx, yy in test_loader:
        user_id = xx[:, 0].long().to(device)
        track_id = xx[:, 1].long().to(device)

        user_country = xx[:, user_country_indices].long().to(device)
        artist_country = xx[:, artist_country_indices].long().to(device)

        outputs = model(user_id, track_id, user_country, artist_country)
        loss = criterion(outputs.float(), yy.float().to(device))
        total_test_loss += loss.item()
        diff = abs(outputs - yy.float().to(device))
        diff_list.append(diff)
        wandb.log({'diff mean': diff.mean().item()})
        wandb.log({'diff std': diff.std().item()})
    wandb.log({'test_loss': total_test_loss / len(test_loader)})
    
mean_diff = torch.cat(diff_list).mean().item()
std_diff = torch.cat(diff_list).std().item()

print(f'Test Loss: {total_test_loss / len(test_loader)}')
print(f'Mean Difference: {mean_diff}')
print(f'Standard Deviation of Difference: {std_diff}')

wandb.finish()

# the two papers (demographic info helps?, globalization?)

# embedding of ids of different people
# also learning a model w/o the demog

In [ ]:
def predict(user_id, artist_id, user_country, artist_country): # move this into the class of model later

    user_id = torch.Tensor(user_id).long()
    artist_id = torch.Tensor(artist_id).long()

    user_country_idx = x.columns.get_loc('user_country_' + user_country)
    artist_country_idx = x.columns.get_loc('artist_country_' + artist_country)
    
    user_country_tensor = torch.tensor(user_country_idx).unsqueeze(0)
    artist_country_tensor = torch.tensor(artist_country_idx).unsqueeze(0)
    
    print(user_id.shape, artist_id.shape, user_country_tensor.shape, artist_country_tensor.shape)
    return user_id, artist_id, user_country_tensor, artist_country_tensor
# gr ca
loaded_model(*predict(0, 0, 'GR', 'CA'))

torch.Size([0]) torch.Size([0]) torch.Size([1]) torch.Size([1])


RuntimeError: Tensors must have same number of dimensions: got 2 and 1

In [ ]:
model_path = 'results/NCF.pth'
torch.save(model, model_path)

loaded_model = torch.load(model_path)
model_weights = loaded_model.state_dict()

In [1]:
print('a')

a


In [4]:
import pandas as pd
import os

DATASET_PATH = '/home/mila/a/armin.moradi/CulturalDiscoverability/ProtoMF/data/lfm2b-1mon'
interactions = pd.read_csv(os.path.join(DATASET_PATH, 'listening_history_train.csv'))
interactions.head()

,old_user_id,old_item_id,timestamp,user_id,item_id
0,2,30937487,2020-02-20 07:10:29,1980,11859
1,2,25099053,2020-02-20 19:00:33,1980,26376
2,2,13081182,2020-02-20 22:27:40,1980,33715
3,2,18298303,2020-02-20 22:37:26,1980,37099
4,2,38104053,2020-02-20 22:42:44,1980,35117


In [37]:
import numpy as np
import pandas as pd
import os 
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
import argparse
import wandb
import time


DATASET_PATH = '/home/mila/a/armin.moradi/ProtoMF/data/lfm2b-1mon'
ITEM_FRACTION = 1.0
USER_FRACTION = 1.0
BATCH_SIZE = 32
LEARNING_RATE = 0.001
EPOCHS = 10
HIDDEN_SIZE = [16, 32, 64]
EMBEDDING_SIZE = 8



DATASET_PATH = '/home/mila/a/armin.moradi/CulturalDiscoverability/ProtoMF/data/lfm2b-1mon'
interactions = pd.read_csv(os.path.join(DATASET_PATH, 'listening_history_train.csv'))
user_demographics = pd.read_csv(os.path.join(DATASET_PATH, 'user_ids.csv'))
item_demographics = pd.read_csv(os.path.join(DATASET_PATH, 'item_ids.csv'))

print('Loaded Data', 'interactions:', len(interactions), 'users', len(user_demographics), 'items', len(item_demographics))
print('-'*50)

Loaded Data interactions: 1485540 users 8418 items 63076
--------------------------------------------------


In [39]:
interactions = interactions[['user_id', 'item_id']]
interactions['count'] = 1
interactions.head()

,user_id,item_id,count
0,1980,11859,1
1,1980,26376,1
2,1980,33715,1
3,1980,37099,1
4,1980,35117,1


In [33]:
item_demographics.head()

,item_id,old_item_id
0,0,17944951
1,1,30711512
2,2,18610221
3,3,18744029
4,4,11059050


In [34]:
user_demographics.head()

,user_id,old_user_id
0,0,112165
1,1,4503
2,2,26487
3,3,29124
4,4,10084


In [41]:
# user_demographics['new_user_id'] = user_demographics.index
# user_demographics = user_demographics.sample(frac=USER_FRACTION, random_state=99) # TODO: CHANGE TO COMPLETE
# print('Sampled User Demographics:', len(user_demographics))
      
# df_final = interactions.merge(user_demographics, left_on='user_id', right_on='new_user_id', how='inner', )

# print(len(df_final), 'len of sampled user interactions')
# # add random negative samples

df_final = interactions

In [42]:
df_final.head()

,user_id,item_id,count
0,1980,11859,1
1,1980,26376,1
2,1980,33715,1
3,1980,37099,1
4,1980,35117,1


In [43]:
user_ids = df_final['user_id'].unique()
item_ids = interactions['item_id'].unique()

negative_samples = []
for user_id in user_ids:
    user_interacted_items = interactions[interactions['user_id'] == user_id]['item_id'].values
    user_negative_items = np.setdiff1d(item_ids, user_interacted_items)
    user_negative_items = np.random.choice(user_negative_items, size=len(user_interacted_items), replace=False)
    for negative_item in user_negative_items:
        negative_samples.append([user_id, negative_item, 0])

negative_interactions = pd.DataFrame(negative_samples, columns=['user_id', 'item_id', 'count'])
# negative_df = negative_interactions.merge(user_demographics, on='user_id', how='inner')
df_final = pd.concat([df_final, negative_interactions], ignore_index=True)

print('Added Negative Samples', '~ Added Items:', len(df_final[df_final['count'] == 0]))
print('-'*50)

Added Negative Samples ~ Added Items: 1485540
--------------------------------------------------


In [47]:
 # drop old_user_id 
df_final = df_final.drop(columns=['old_user_id'])
df_final.head()

,user_id,item_id,count
0,1980,11859,1
1,1980,26376,1
2,1980,33715,1
3,1980,37099,1
4,1980,35117,1


In [19]:
# item_demographics['new_item_id'] = item_demographics.index
# item_demographics = item_demographics[item_demographics['item_id'].isin(interactions['item_id'])]
# item_demographics = item_demographics.sample(frac=ITEM_FRACTION, random_state=99) #TODO: CHANGE TO COMPLETE
# print('Sampled Item Demographics:', len(item_demographics))

# df_final = df_final.merge(item_demographics, on='item_id', how='inner')

# print('DF FINAL - n rows:', len(df_final))
# print('-'*50)

# df_final.head()

Sampled Item Demographics: 63076
DF FINAL - n rows: 2971080
--------------------------------------------------


,old_user_id_x,old_item_id_x,timestamp,user_id,item_id,old_user_id_y,new_user_id,count,old_user_id,old_item_id_y,new_track_id,new_item_id
0,2.0,30937487.0,2020-02-20 07:10:29,1980,11859,2.0,1980,NaN,NaN,30937487,11859,11859
1,66.0,30937487.0,2020-03-19 22:07:55,1005,11859,66.0,1005,NaN,NaN,30937487,11859,11859
2,590.0,30937487.0,2020-03-14 13:55:10,116,11859,590.0,116,NaN,NaN,30937487,11859,11859
3,2035.0,30937487.0,2020-03-09 12:56:19,2899,11859,2035.0,2899,NaN,NaN,30937487,11859,11859
4,3109.0,30937487.0,2020-02-21 15:33:26,4063,11859,3109.0,4063,NaN,NaN,30937487,11859,11859


In [48]:
df_final.tail()

,user_id,item_id,count
2971075,8113,33845,0
2971076,8113,7445,0
2971077,8113,16254,0
2971078,8113,6909,0
2971079,8113,1188,0


In [49]:
# add up the columns with repeated user_id and item_id and count
df_final = df_final.groupby(['user_id', 'item_id']).sum().reset_index()
df_final.head()

,user_id,item_id,count
0,0,0,1
1,0,38,0
2,0,51,0
3,0,65,1
4,0,86,0


,user_id,item_id,count


In [ ]:

labels = ['0-18', '18-30', '30-50', '50+']
bins = [0, 18, 30, 50, 100]
df_final['user_age'] = pd.cut(df_final['user_age'], bins=bins, labels=labels, right=False)

# replace user id with new_user_id and delete user_id
df_final = df_final.drop(columns=['user_id'])
df_final = df_final.rename(columns={'new_user_id':'user_id'})

# replace user id with new_user_id and delete user_id
df_final = df_final.drop(columns=['track_id'])
df_final = df_final.rename(columns={'new_track_id':'track_id'})

cols = df_final.columns.tolist()
cols = [cols[-1], cols[3]] + cols[0:3] + cols[4:-1]
df_final = df_final[cols]
print('Finished Pre-processing')
print('-'*50)


# shape data


In [56]:
y = df_final['count']
x = df_final.drop(['count'], axis=1)
encoder = OneHotEncoder()
# columns_to_encode = x.columns[2:]
# encoder.fit(x[columns_to_encode])
# encoded_x = encoder.transform(x[columns_to_encode])
# encoded_x = pd.DataFrame(encoded_x, columns=encoder.get_feature_names_out(columns_to_encode))
# x = pd.concat([x[['user_id', 'track_id']], encoded_x], axis=1)

# user_country_indices = [i for i, col in enumerate(x.columns) if col.startswith('user_country_')]
# artist_country_indices = [i for i, col in enumerate(x.columns) if col.startswith('artist_country_')]
# user_gender_indices = [i for i, col in enumerate(x.columns) if col.startswith('user_gender_')]
# artist_gender_indices = [i for i, col in enumerate(x.columns) if col.startswith('artist_gender')]
# user_age_indices = [i for i, col in enumerate(x.columns) if col.startswith('user_age_')]
# unique_user_ids, unique_track_ids = max(x['user_id']) + 1, max(x['track_id']) + 1

# split the interactions of each user into train, val, and test (80-10-10)

train_share = 0.75
val_share = 0.15
test_share = 1 - train_share - val_share

user_interactions = {}
for i in range(len(x)):
    user_id = x.iloc[i]['user_id']
    if user_id not in user_interactions:
        user_interactions[user_id] = []
    user_interactions[user_id].append(i)

train_indices, val_indices, test_indices = [], [], []
for user_id, indices in user_interactions.items():
    np.random.shuffle(indices)
    train_indices += indices[:int(train_share * len(indices))]
    val_indices += indices[int(train_share * len(indices)):int((train_share + val_share ) * len(indices))]
    test_indices += indices[int((1 - test_share) * len(indices)):]
    
train_x, train_y = x.iloc[train_indices], y.iloc[train_indices]
val_x, val_y = x.iloc[val_indices], y.iloc[val_indices]
test_x, test_y = x.iloc[test_indices], y.iloc[test_indices]


train_x.head()

,user_id,item_id
660,0,44802
720,0,48803
639,0,43312
145,0,8281
280,0,16501


In [58]:

# create dataloaders

train_dataset = TensorDataset(torch.tensor(train_x.values, dtype=torch.float), torch.tensor(train_y.values, dtype=torch.float))
val_dataset = TensorDataset(torch.tensor(val_x.values, dtype=torch.float), torch.tensor(val_y.values, dtype=torch.float))
test_dataset = TensorDataset(torch.tensor(test_x.values, dtype=torch.float), torch.tensor(test_y.values, dtype=torch.float))

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)
print(f'created loaders with batchsize {BATCH_SIZE}', len(train_dataset), len(val_dataset), len(test_dataset))

created loaders with batchsize 32 2226217 444421 300442


In [60]:
# set up wandb logging

wandb.init(project='cultural-rs', entity='armornine')
print('Initialized WandB')

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f8af8e06d30>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f8ad5959160, raw_cell="# set up wandb logging

wandb.init(project='cultur.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bcn-h002.server.mila.quebec/home/mila/a/armin.moradi/CulturalDiscoverability/notebooks/evaluate_baseline.ipynb#X43sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


TypeError: _resume_backend() takes 1 positional argument but 2 were given

wandb: wandb version 0.16.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Initialized WandB


In [62]:
unique_user_ids = df_final['user_id'].nunique()
unique_track_ids = df_final['item_id'].nunique()

In [67]:
class AgnosticNCF(nn.Module):
    def __init__(self, num_users, num_tracks, hidden_size=[16, 32, 32]): # 256, 128, 64
        super(AgnosticNCF, self).__init__()

        self.embedding_size = 8 # 32

        self.user_id_embedding = nn.Embedding(num_users, self.embedding_size)
        self.track_id_embedding = nn.Embedding(num_tracks, self.embedding_size)

        self.fc_layers = nn.Sequential(
            nn.Linear(self.embedding_size * 2, hidden_size[0]),
            nn.ReLU(),
            nn.Linear(hidden_size[0], hidden_size[1]),
            nn.ReLU(),
            nn.Linear(hidden_size[1], hidden_size[2]),
            nn.ReLU(),
            nn.Linear(hidden_size[2], 1),
        )

    def forward(self, user_id, artist_id):

        user_id_embeds = self.user_id_embedding(user_id)
        track_id_embeds = self.track_id_embedding(artist_id)
        

        concatenated = torch.cat([user_id_embeds, track_id_embeds], dim=1)
        output = self.fc_layers(concatenated.float())

        return output.squeeze()

print('Start Training')
print('-'*50)

start_time = time.time()
model = AgnosticNCF(num_users=unique_user_ids, num_tracks=unique_track_ids, hidden_size=[64, 128, 64]).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

val_loss = []
train_loss = []


for epoch in tqdm(range(EPOCHS), desc='Epochs'):
    model.train()
    total_loss = 0.0
    for xx, yy in train_loader:
        user_id = xx[:, 0].long().to(device)
        track_id = xx[:, 1].long().to(device)

        # user_country = xx[:, user_country_indices].long().to(device)
        # artist_country = xx[:, artist_country_indices].long().to(device)

        optimizer.zero_grad()
        outputs = model(user_id, track_id)# , user_country, artist_country)
        loss = criterion(outputs.float(), yy.float().to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    wandb.log({'train_loss': total_loss / len(train_loader)})
    
    model.eval()
    total_val_loss = 0.0
    for xx, yy in val_loader:
        user_id = xx[:, 0].long().to(device)
        track_id = xx[:, 1].long().to(device)

        # user_country = xx[:, user_country_indices].long().to(device)
        # artist_country = xx[:, artist_country_indices].long().to(device)

        outputs = model(user_id, track_id)
        loss = criterion(outputs.float(), yy.float().to(device))
        total_val_loss += loss.item()
    wandb.log({'val_loss': total_val_loss / len(val_loader)})
        
    print(f'Epoch {epoch + 1}/{EPOCHS}, Loss: {total_loss / len(train_loader)}, Val Loss: {total_val_loss / len(val_loader)}')
    train_loss.append(total_loss / len(train_loader))
    val_loss.append(total_val_loss / len(val_loader))

    torch.save(model.state_dict(), LOGS_PATH + 'model_weights.pth')

print('Training Time:', round((time.time() - start_time)/60, 2), 'minutes')
print('-'*50)

# test

model.eval()
total_test_loss = 0.0
diff_list = []

with torch.no_grad():
    for xx, yy in test_loader:
        user_id = xx[:, 0].long().to(device)
        track_id = xx[:, 1].long().to(device)

        # user_country = xx[:, user_country_indices].long().to(device)
        # artist_country = xx[:, artist_country_indices].long().to(device)

        outputs = model(user_id, track_id)
        loss = criterion(outputs.float(), yy.float().to(device))
        total_test_loss += loss.item()
        diff = abs(outputs - yy.float().to(device))
        diff_list.append(diff)
        wandb.log({'diff mean': diff.mean().item()})
        wandb.log({'diff std': diff.std().item()})
    wandb.log({'test_loss': total_test_loss / len(test_loader)})
    
mean_diff = torch.cat(diff_list).mean().item()
std_diff = torch.cat(diff_list).std().item()

print(f'Test Loss: {total_test_loss / len(test_loader)}')
print(f'Mean Difference: {mean_diff}')
print(f'Standard Deviation of Difference: {std_diff}')

wandb.finish()

Start Training
--------------------------------------------------


Epochs:   0%|          | 0/10 [03:26<?, ?it/s]


NameError: name 'user_country' is not defined

In [79]:
import numpy as np
import pandas as pd
import os 
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
import wandb
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

DATASET_PATH = '/home/mila/a/armin.moradi/ProtoMF/data/lfm2b-1mon'
ITEM_FRACTION = 1.0
USER_FRACTION = 1.0
BATCH_SIZE = 32
LEARNING_RATE = 0.001
EPOCHS = 10
HIDDEN_SIZE = [16, 32, 64]
EMBEDDING_SIZE = 8



DATASET_PATH = '/home/mila/a/armin.moradi/CulturalDiscoverability/ProtoMF/data/lfm2b-1mon'
interactions = pd.read_csv(os.path.join(DATASET_PATH, 'listening_history_train.csv'))
user_demographics = pd.read_csv(os.path.join(DATASET_PATH, 'user_ids.csv'))
item_demographics = pd.read_csv(os.path.join(DATASET_PATH, 'item_ids.csv'))

print('Loaded Data', 'interactions:', len(interactions), 'users', len(user_demographics), 'items', len(item_demographics))
print('-'*50)

interactions = interactions[['user_id', 'item_id']]
interactions['count'] = 1
interactions.head()

# take only 10 interactions
# interactions = interactions.sample(frac=0.01, random_state=42) #TODO REMOVE


print('Sampled Interactions', len(interactions))
df_final = interactions

user_ids = df_final['user_id'].unique()
item_ids = df_final['item_id'].unique()

negative_samples = []
for user_id in user_ids:
    user_interacted_items = interactions[interactions['user_id'] == user_id]['item_id'].values
    user_negative_items = np.setdiff1d(item_ids, user_interacted_items)
    user_negative_items = np.random.choice(user_negative_items, size=len(user_interacted_items), replace=False)
    for negative_item in user_negative_items:
        negative_samples.append([user_id, negative_item, 0])

negative_interactions = pd.DataFrame(negative_samples, columns=['user_id', 'item_id', 'count'])
# negative_df = negative_interactions.merge(user_demographics, on='user_id', how='inner')
df_final = pd.concat([df_final, negative_interactions], ignore_index=True)

print('Added Negative Samples', '~ Added Items:', len(df_final[df_final['count'] == 0]))
print('-'*50)

 # drop old_user_id 
# df_final = df_final.drop(columns=['old_user_id'])

# add up the columns with repeated user_id and item_id and count
df_final = df_final.groupby(['user_id', 'item_id']).sum().reset_index()
print('FINAL DF', df_final.head())

y = df_final['count']
x = df_final.drop(['count'], axis=1)

print('x', x.head())
print('y', y.head())

encoder = OneHotEncoder()
# columns_to_encode = x.columns[2:]
# encoder.fit(x[columns_to_encode])
# encoded_x = encoder.transform(x[columns_to_encode])
# encoded_x = pd.DataFrame(encoded_x, columns=encoder.get_feature_names_out(columns_to_encode))
# x = pd.concat([x[['user_id', 'track_id']], encoded_x], axis=1)

# user_country_indices = [i for i, col in enumerate(x.columns) if col.startswith('user_country_')]
# artist_country_indices = [i for i, col in enumerate(x.columns) if col.startswith('artist_country_')]
# user_gender_indices = [i for i, col in enumerate(x.columns) if col.startswith('user_gender_')]
# artist_gender_indices = [i for i, col in enumerate(x.columns) if col.startswith('artist_gender')]
# user_age_indices = [i for i, col in enumerate(x.columns) if col.startswith('user_age_')]
# unique_user_ids, unique_track_ids = max(x['user_id']) + 1, max(x['track_id']) + 1

# split the interactions of each user into train, val, and test (80-10-10)

train_share = 0.75
val_share = 0.15
test_share = 1 - train_share - val_share

user_interactions = {}
for i in range(len(x)):
    user_id = x.iloc[i]['user_id']
    if user_id not in user_interactions:
        user_interactions[user_id] = []
    user_interactions[user_id].append(i)

train_indices, val_indices, test_indices = [], [], []
for user_id, indices in user_interactions.items():
    np.random.shuffle(indices)
    train_indices += indices[:int(train_share * len(indices))]
    val_indices += indices[int(train_share * len(indices)):int((train_share + val_share ) * len(indices))]
    test_indices += indices[int((1 - test_share) * len(indices)):]
    
train_x, train_y = x.iloc[train_indices], y.iloc[train_indices]
val_x, val_y = x.iloc[val_indices], y.iloc[val_indices]
test_x, test_y = x.iloc[test_indices], y.iloc[test_indices]

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f8af8e03f10>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f8acb89b430, raw_cell="import numpy as np
import pandas as pd
import os 
.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bcn-h002.server.mila.quebec/home/mila/a/armin.moradi/CulturalDiscoverability/notebooks/evaluate_baseline.ipynb#X45sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


TypeError: _resume_backend() takes 1 positional argument but 2 were given

Loaded Data interactions: 1485540 users 8418 items 63076
--------------------------------------------------
Sampled Interactions 1485540
Added Negative Samples ~ Added Items: 1485540
--------------------------------------------------
FINAL DF    user_id  item_id  count
0        0        0      1
1        0       65      1
2        0       98      1
3        0      109      1
4        0      124      1
x    user_id  item_id
0        0        0
1        0       65
2        0       98
3        0      109
4        0      124
y 0    1
1    1
2    1
3    1
4    1
Name: count, dtype: int64
Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f8af8e03f10>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f8acb89b3d0, execution_count=79 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f8acb89b430, raw_cell="import numpy as np
import pandas as pd
import os 
.." store_history=True silent=False shell

TypeError: _pause_backend() takes 1 positional argument but 2 were given

In [ ]:

train_x.head()


print(len(train_x), len(val_x), len(test_x))
# create dataloaders

train_dataset = TensorDataset(torch.tensor(train_x.values, dtype=torch.float), torch.tensor(train_y.values, dtype=torch.float))
val_dataset = TensorDataset(torch.tensor(val_x.values, dtype=torch.float), torch.tensor(val_y.values, dtype=torch.float))
test_dataset = TensorDataset(torch.tensor(test_x.values, dtype=torch.float), torch.tensor(test_y.values, dtype=torch.float))

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)
print(f'created loaders with batchsize {BATCH_SIZE}', len(train_dataset), len(val_dataset), len(test_dataset))

# set up wandb logging

wandb.init(project='fooling-around', entity='armornine')
print('Initialized WandB')

unique_user_ids = df_final['user_id'].nunique()
unique_track_ids = df_final['item_id'].nunique()

class AgnosticNCF(nn.Module):
    def __init__(self, num_users, num_tracks, hidden_size=[16, 32, 32]): # 256, 128, 64
        super(AgnosticNCF, self).__init__()

        self.embedding_size = 8 # 32

        self.user_id_embedding = nn.Embedding(num_users, self.embedding_size)
        self.track_id_embedding = nn.Embedding(num_tracks, self.embedding_size)

        self.fc_layers = nn.Sequential(
            nn.Linear(self.embedding_size * 2, hidden_size[0]),
            nn.ReLU(),
            nn.Linear(hidden_size[0], hidden_size[1]),
            nn.ReLU(),
            nn.Linear(hidden_size[1], hidden_size[2]),
            nn.ReLU(),
            nn.Linear(hidden_size[2], 1),
        )

    def forward(self, user_id, artist_id):

        user_id_embeds = self.user_id_embedding(user_id)
        track_id_embeds = self.track_id_embedding(artist_id)
        

        concatenated = torch.cat([user_id_embeds, track_id_embeds], dim=1)
        output = self.fc_layers(concatenated.float())

        return output.squeeze()

print('Start Training')
print('-'*50)

start_time = time.time()
model = AgnosticNCF(num_users=unique_user_ids, num_tracks=unique_track_ids, hidden_size=[64, 128, 64]).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

val_loss = []
train_loss = []


for epoch in tqdm(range(EPOCHS), desc='Epochs'):
    model.train()
    total_loss = 0.0
    for xx, yy in train_loader:
        user_id = xx[:, 0].long().to(device)
        track_id = xx[:, 1].long().to(device)

        # user_country = xx[:, user_country_indices].long().to(device)
        # artist_country = xx[:, artist_country_indices].long().to(device)

        optimizer.zero_grad()
        outputs = model(user_id, track_id)# , user_country, artist_country)
        loss = criterion(outputs.float(), yy.float().to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    wandb.log({'train_loss': total_loss / len(train_loader)})
    
    model.eval()
    total_val_loss = 0.0
    for xx, yy in val_loader:
        user_id = xx[:, 0].long().to(device)
        track_id = xx[:, 1].long().to(device)

        # user_country = xx[:, user_country_indices].long().to(device)
        # artist_country = xx[:, artist_country_indices].long().to(device)

        outputs = model(user_id, track_id)
        loss = criterion(outputs.float(), yy.float().to(device))
        total_val_loss += loss.item()
    wandb.log({'val_loss': total_val_loss / len(val_loader)})
        
    print(f'Epoch {epoch + 1}/{EPOCHS}, Loss: {total_loss / len(train_loader)}, Val Loss: {total_val_loss / len(val_loader)}')
    train_loss.append(total_loss / len(train_loader))
    val_loss.append(total_val_loss / len(val_loader))

    torch.save(model.state_dict(), LOGS_PATH + 'model_weights.pth')

print('Training Time:', round((time.time() - start_time)/60, 2), 'minutes')
print('-'*50)

# test

model.eval()
total_test_loss = 0.0
diff_list = []

with torch.no_grad():
    for xx, yy in test_loader:
        user_id = xx[:, 0].long().to(device)
        track_id = xx[:, 1].long().to(device)

        # user_country = xx[:, user_country_indices].long().to(device)
        # artist_country = xx[:, artist_country_indices].long().to(device)

        outputs = model(user_id, track_id)
        loss = criterion(outputs.float(), yy.float().to(device))
        total_test_loss += loss.item()
        diff = abs(outputs - yy.float().to(device))
        diff_list.append(diff)
        wandb.log({'diff mean': diff.mean().item()})
        wandb.log({'diff std': diff.std().item()})
    wandb.log({'test_loss': total_test_loss / len(test_loader)})
    
mean_diff = torch.cat(diff_list).mean().item()
std_diff = torch.cat(diff_list).std().item()

print(f'Test Loss: {total_test_loss / len(test_loader)}')
print(f'Mean Difference: {mean_diff}')
print(f'Standard Deviation of Difference: {std_diff}')

wandb.finish()